In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
import pandas as pd
from sklearn.inspection import permutation_importance


In [2]:
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
test_df = pd.read_csv('../data/raw/dmml1_test.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [3]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
test_df = pd.merge(test_df,store,how="inner",on="Store ID")

In [4]:
train_df

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,4,2015-04-30,6884,716,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,1,3,2015-04-29,6764,756,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,1,2,2015-04-28,6861,678,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
3,1,1,2015-04-27,6523,647,1,1,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,1,7,2015-04-26,0,0,0,0,0,0,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
246899,300,5,2013-01-04,5524,646,1,0,0,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
246900,300,4,2013-01-03,5563,718,1,0,0,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
246901,300,3,2013-01-02,6218,753,1,0,0,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN


In [5]:
store

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,2,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
3,4,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,5,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
295,296,a,a,690.0,6.0,2007.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
297,298,d,c,1340.0,10.0,2006.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
298,299,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [6]:
train_df["CompetitionDistance"].replace(np.nan,train_df["CompetitionDistance"].mean(),inplace=True)
train_df.fillna(0,inplace=True)

In [7]:
train_df["StateHoliday"].replace("0",0,inplace=True)

In [8]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week
train_df['Weekend'] = np.where(train_df['DayOfWeek'].isin([6, 7]), 1, 0)  # Samstag = 6, Sonntag = 7

train_df['Quarter'] = train_df['Date'].dt.quarter
train_df['DayOfYear'] = train_df['Date'].dt.dayofyear
train_df['DayOfMonth'] = train_df['Date'].dt.day

train_df['Season'] = train_df['Month'].apply(lambda month: (month%12 // 3 + 1))
train_df['Season'].replace(to_replace=[1,2,3,4], value=['Winter', 'Frühling','Sommer','Herbst'], inplace=True)

# Entfernen der ursprünglichen Date-Spalte
train_df.drop('Date', axis=1, inplace=True)

In [9]:
train_df["StateHoliday"] = train_df["StateHoliday"].replace("a","Feiertag").replace("b","Ostern").replace("c","Weihnachten").replace(0,"Kein Feiertag")

In [10]:
train_df["StoreType"] = train_df["StoreType"].replace("a","StoreType A").replace("b","StoreTypee B").replace("c","StoreType C").replace("d","StoreType D")

In [11]:
train_df["Assortment"] = train_df["Assortment"].replace("a","Basic").replace("b","Extra").replace("c","Extended")

In [12]:
train_df['PromoInterval'].unique()

array(['Jan,Apr,Jul,Oct', 0, 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec'],
      dtype=object)

In [13]:
train_df["PromoInterval"] = train_df["PromoInterval"].replace(0,"No PromoInterval") #.replace("Jan,Apr,Jul,Oct","From Jan to Oct").replace("Feb,May,Aug,Nov","From Feb to Nov").replace("Mar,Jun,Sept,Dec","From Mar to Dec")

# def myPinterval(x):
#     if x=='From Feb to Nov':
#         return([0,1,0,0,1,0,0,1,0,0,1,0])
#     elif x=='From Jan to Oct':
#         return([1,0,0,1,0,0,1,0,0,1,0,0])
#     elif x== 'From Mar to Dec':
#         return([0,0,1,0,0,1,0,0,1,0,0,1])
#     else:
#         return(np.repeat(0,12).tolist())

# proInt = train_df.PromoInterval.apply(myPinterval).tolist()
# proInt = pd.DataFrame(proInt, columns = ['PromoInt'+ str(i) for i in range(1,13)])
# train_df = train_df.drop('PromoInterval',1).join(proInt)

In [14]:
# Erstellen Sie eine temporäre Spalte, die 1 ist, wenn der Tag ein Feiertag ist, und 0 sonst
train_df['Temp'] = np.where(train_df['StateHoliday'] != 'Kein Feiertag', 1, 0)

# Erstellen Sie die Spalte "TagVorFeiertag", die 1 ist, wenn der nächste Tag ein Feiertag ist, und 0 sonst
train_df['TagVorFeiertag'] = train_df['Temp'].shift(-1).fillna(0)

# Erstellen Sie die Spalte "TagnachFeiertag", die 1 ist, wenn der vorherige Tag ein Feiertag war, und 0 sonst
train_df['TagnachFeiertag'] = train_df['Temp'].shift(1).fillna(0)

# Entfernen Sie die temporäre Spalte
train_df.drop('Temp', axis=1, inplace=True)

In [15]:
# Erstellen Sie den OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Liste der Spalten, die encodiert werden sollen
columns_to_encode = ['StateHoliday', 'StoreType', 'Assortment', 'Season', 'PromoInterval']

for column in columns_to_encode:
    # Führen Sie das One-Hot-Encoding für die Spalte durch
    column_encoded = encoder.fit_transform(train_df[[column]])

    # Erstellen Sie einen DataFrame aus dem encodierten Array
    column_df = pd.DataFrame(column_encoded, columns=encoder.categories_[0])

    # Fügen Sie die neuen Spalten zum DataFrame hinzu und entfernen Sie die alte Spalte
    train_df = pd.concat([train_df.drop(column, axis=1), column_df], axis=1)

In [16]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

In [17]:
X

,Store ID,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,...,Extended,Extra,Frühling,Herbst,Sommer,Winter,"Feb,May,Aug,Nov","Jan,Apr,Jul,Oct","Mar,Jun,Sept,Dec",No PromoInterval
0,1,4,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,3,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,2,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,1,1,1,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,7,0,0,0,570.0,11.0,2007.0,1,13.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,1,0,0,9260.0,0.0,0.0,0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
246899,300,5,1,0,1,9260.0,0.0,0.0,0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
246900,300,4,1,0,1,9260.0,0.0,0.0,0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
246901,300,3,1,0,1,9260.0,0.0,0.0,0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [19]:
models=[LinearRegression(), 
        Ridge(), 
        Lasso(), 
        DecisionTreeRegressor(random_state=42), 
        GradientBoostingRegressor(random_state=42), 
        RandomForestRegressor(n_estimators=42, n_jobs=-1, random_state=42)]

model_names=["LinearRegression","Ridge","Lasso", "DecisionTreeRegressor", "GradientBoostingRegressor", "RandomForestRegressor"]

MSE=[]
r_2=[]
accuracy = []
RMSE = []
training_time = []
d={}

for index in range (len(models)):
    model=models[index]
    start_time = time.time()
    model.fit(X_train,y_train)
    end_time = time.time()
    y_pre=model.predict(X_test)

    MSE.append(round(mean_squared_error(y_true=y_test,y_pred=y_pre),5))
    r_2.append(r2_score(y_true=y_test,y_pred=y_pre))
    accuracy.append((model.score(X_test,y_test))*100)
    RMSE.append(sqrt(mean_squared_error(y_true=y_test,y_pred=y_pre)))
    training_time.append(end_time - start_time)
    print(f'{model_names[index]} wurde trainiert und benötigte {end_time - start_time} Sekunden')

d=pd.DataFrame({'Modelling Name':model_names,'MSE':MSE,"R_2":r_2,"Accuracy":accuracy,"RMSE":RMSE, "Training Time": training_time})
print("Verwendete Features: ", X_train.columns.tolist())

LinearRegression wurde trainiert und benötigte 0.40044474601745605 Sekunden
Ridge wurde trainiert und benötigte 0.12184619903564453 Sekunden


c:\Users\Wladi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.473e+11, tolerance: 3.164e+08
  model = cd_fast.enet_coordinate_descent(


Lasso wurde trainiert und benötigte 13.915088415145874 Sekunden
DecisionTreeRegressor wurde trainiert und benötigte 3.4902732372283936 Sekunden
GradientBoostingRegressor wurde trainiert und benötigte 42.655927658081055 Sekunden
RandomForestRegressor wurde trainiert und benötigte 27.18661332130432 Sekunden
Verwendete Features:  ['Store ID', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Day', 'WeekOfYear', 'Weekend', 'Quarter', 'DayOfYear', 'DayOfMonth', 'TagVorFeiertag', 'TagnachFeiertag', 'Feiertag', 'Kein Feiertag', 'Ostern', 'Weihnachten', 'StoreType A', 'StoreType C', 'StoreType D', 'StoreTypee B', 'Basic', 'Extended', 'Extra', 'Frühling', 'Herbst', 'Sommer', 'Winter', 'Feb,May,Aug,Nov', 'Jan,Apr,Jul,Oct', 'Mar,Jun,Sept,Dec', 'No PromoInterval']


In [20]:
d

,Modelling Name,MSE,R_2,Accuracy,RMSE,Training Time
0,LinearRegression,7.170746e+06,0.557781,55.778109,2677.824885,0.400445
1,Ridge,7.171862e+06,0.557712,55.771230,2678.033159,0.121846
2,Lasso,7.174501e+06,0.557550,55.754952,2678.525923,13.915088
3,DecisionTreeRegressor,1.062960e+06,0.934447,93.444740,1030.999588,3.490273
4,GradientBoostingRegressor,4.513986e+06,0.721623,72.162309,2124.614420,42.655928
5,RandomForestRegressor,6.417690e+05,0.960422,96.042219,801.104887,27.186613


In [21]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

models=[LinearRegression(), 
        Ridge(), 
        Lasso(), 
        DecisionTreeRegressor(random_state=42), 
        GradientBoostingRegressor(random_state=42), 
        RandomForestRegressor(n_estimators=42, n_jobs=-1, random_state=42)]

model_names=["LinearRegression","Ridge","Lasso", "DecisionTreeRegressor", "GradientBoostingRegressor", "RandomForestRegressor"]

MSE=[]
r_2=[]
accuracy = []
RMSE = []
training_time = []
d={}
best_features = []

for index in range (len(models)):
    model=models[index]
    
    # Ermitteln der Feature-Importance
    model.fit(X_train, y_train)
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    else:
        importances = np.abs(model.coef_)
        
    columns = list(X_train.columns)
    feature_importance_df = pd.DataFrame({'Features':columns, 'Values':importances})
    feature_importance_df.sort_values(by='Values', ascending=False, inplace=True)
    model_spalten = feature_importance_df.loc[feature_importance_df['Values'] > 0.01].Features.tolist()
    best_features.append(model_spalten)
    
    # Trainieren des Modells mit den wichtigsten Features
    X_train_model = X_train[model_spalten]
    X_test_model = X_test[model_spalten]
    start_time = time.time()
    model.fit(X_train_model, y_train)
    end_time = time.time()
    
    y_pre=model.predict(X_test_model)

    # Berechnung der Metriken
    mse = mean_squared_error(y_test, y_pre)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pre)
    MSE.append(round(mse, 5))
    RMSE.append(round(rmse, 5))
    r_2.append(round(r2, 5))
    accuracy.append((model.score(X_test_model, y_test)) * 100)

    training_time.append(end_time - start_time)
    print(f'{model_names[index]} wurde trainiert und benötigte {end_time - start_time} Sekunden')
    print("Verwendete Features: ", model_spalten)
    mse = mean_squared_error(y_test, y_pre)
    rmse = np.sqrt(mse)  # RMSE ist einfach die Quadratwurzel von MSE
    r2 = r2_score(y_test, y_pre)

    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R² Score: {r2}")
    print()
    print("--------------------------------------------------")
    print()

d=pd.DataFrame({'Modelling Name':model_names,'MSE':MSE,"R_2":r_2,"Accuracy":accuracy,"RMSE":RMSE, "Training Time": training_time, "Best Features": best_features})

LinearRegression wurde trainiert und benötigte 0.3524348735809326 Sekunden
Verwendete Features:  ['No PromoInterval', 'Promo2', 'Jan,Apr,Jul,Oct', 'Mar,Jun,Sept,Dec', 'Feb,May,Aug,Nov', 'Open', 'Month', 'Promo', 'StoreTypee B', 'Weihnachten', 'TagVorFeiertag', 'Kein Feiertag', 'Weekend', 'StoreType D', 'Herbst', 'Winter', 'TagnachFeiertag', 'StoreType A', 'Extended', 'Basic', 'Feiertag', 'StoreType C', 'Quarter', 'DayOfWeek', 'Frühling', 'Sommer', 'Ostern', 'SchoolHoliday', 'Year', 'DayOfYear', 'DayOfMonth', 'Day', 'Extra', 'Promo2SinceYear', 'WeekOfYear', 'Promo2SinceWeek', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'CompetitionDistance', 'Store ID']
Mean Squared Error (MSE): 7048823.457395297
Root Mean Squared Error (RMSE): 2654.962044435908
R² Score: 0.5609691066296956

--------------------------------------------------

Ridge wurde trainiert und benötigte 0.1202242374420166 Sekunden
Verwendete Features:  ['Open', 'No PromoInterval', 'Promo2', 'Month', 'Promo', 'Jan,Ap

c:\Users\Wladi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.358e+11, tolerance: 3.171e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\Wladi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.100e+11, tolerance: 3.171e+08
  model = cd_fast.enet_coordinate_descent(


Lasso wurde trainiert und benötigte 11.81499195098877 Sekunden
Verwendete Features:  ['Open', 'Promo', 'StoreTypee B', 'Weihnachten', 'TagVorFeiertag', 'Kein Feiertag', 'Jan,Apr,Jul,Oct', 'Herbst', 'Weekend', 'Basic', 'TagnachFeiertag', 'Winter', 'Extended', 'Ostern', 'Sommer', 'DayOfWeek', 'Quarter', 'Feb,May,Aug,Nov', 'StoreType C', 'Frühling', 'StoreType D', 'SchoolHoliday', 'Year', 'Month', 'Promo2SinceWeek', 'WeekOfYear', 'Feiertag', 'StoreType A', 'DayOfMonth', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'DayOfYear', 'Day', 'CompetitionOpenSinceYear', 'CompetitionDistance', 'Store ID']
Mean Squared Error (MSE): 7049555.037997065
Root Mean Squared Error (RMSE): 2655.0998169554878
R² Score: 0.5609235406587065

--------------------------------------------------

DecisionTreeRegressor wurde trainiert und benötigte 1.343264102935791 Sekunden
Verwendete Features:  ['Open', 'CompetitionDistance', 'Store ID', 'Promo', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'DayOfWeek', 'Comp

In [22]:
d

,Modelling Name,MSE,R_2,Accuracy,RMSE,Training Time,Best Features
0,LinearRegression,7.048823e+06,0.56097,56.096911,2654.96204,0.352435,"[No PromoInterval, Promo2, Jan,Apr,Jul,Oct, Ma..."
1,Ridge,7.048814e+06,0.56097,56.096967,2654.96035,0.120224,"[Open, No PromoInterval, Promo2, Month, Promo,..."
2,Lasso,7.049555e+06,0.56092,56.092354,2655.09982,11.814992,"[Open, Promo, StoreTypee B, Weihnachten, TagVo..."
3,DecisionTreeRegressor,1.594837e+06,0.90067,90.066675,1262.86844,1.343264,"[Open, CompetitionDistance, Store ID, Promo, C..."
4,GradientBoostingRegressor,4.486954e+06,0.72053,72.053333,2118.24305,22.299670,"[Open, CompetitionDistance, Promo, Promo2Since..."
5,RandomForestRegressor,1.029026e+06,0.93591,93.590788,1014.40900,11.626532,"[Open, CompetitionDistance, Store ID, Promo, C..."


In [23]:
fsdf.fashe + 1

NameError: name 'fsdf' is not defined

# RandomForestRegressor

In [ ]:
X = train_df.drop(["Sales","Customers"],axis=1)
y = train_df["Sales"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

forest = RandomForestRegressor(random_state=42)
forest.fit(X_train, y_train)
importances = forest.feature_importances_

columns = list(X_train.columns)
feature_importance_df = pd.DataFrame({'Features':columns, 'Values':importances})
feature_importance_df.sort_values(by='Values', ascending=False, inplace=True)
model_spalten = feature_importance_df.loc[feature_importance_df['Values'] > 0.01].Features.tolist()
print(model_spalten)

X = train_df[model_spalten]
y = train_df["Sales"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

['Open', 'CompetitionDistance', 'Store ID', 'Promo', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'DayOfWeek', 'DayOfYear', 'Basic', 'Extended', 'Promo2SinceWeek']


In [ ]:
# Definieren Sie die Parameter, die Sie optimieren möchten
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [None],
    'min_samples_split': [10]
}

# Erstellen Sie ein neues RandomForestRegressor-Objekt
forest = RandomForestRegressor(random_state=42)

# Erstellen Sie das GridSearchCV-Objekt
grid_search = GridSearchCV(forest, param_grid, cv=5, n_jobs=-1)

# Führen Sie die Suche aus
grid_search.fit(X_train, y_train)

# Drucken Sie die besten Parameter aus
print(grid_search.best_params_)


{'max_depth': None, 'min_samples_split': 10, 'n_estimators': 300}


In [ ]:
# # Definieren Sie die optimierten Parameter
best_params = {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 300}

# Erstellen Sie ein neues RandomForestRegressor-Objekt mit den optimierten Parametern
forest = RandomForestRegressor(max_depth=best_params['max_depth'], 
                                min_samples_split=best_params['min_samples_split'], 
                                n_estimators=best_params['n_estimators'],
                                random_state=42,
                                n_jobs=-1)
forest = RandomForestRegressor(random_state=42)
# Trainieren Sie das Modell mit den Trainingsdaten
forest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Machen Sie Vorhersagen auf den Testdaten
y_pred = forest.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # RMSE ist einfach die Quadratwurzel von MSE
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 997235.3682094166
Root Mean Squared Error (RMSE): 998.616727383142
R² Score: 0.9372307752280711
